<a href="https://colab.research.google.com/github/samanthasu12/Contentstack_Internship/blob/main/L%26DDataGraphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to my L&D Graphs Project!**
----
Here are the steps that you need to take to graph the data.
1. Make sure that L&D Data Calculation is run prior to this colab, as this takes the finalized sheet and runs it
2. Make sure that the sheet, "Course A", "For Sam - Data Dreams (Android Sheep)" is filled with Data
3. After you've completed your checks, please click Runtime (located at the very top of the colab) and Run All. It will then ask you to connect the google colab to your own google account, and you just need to sign in.
4. This colab notebook should take around 10 seconds to completely automate. All the graphs should be found below the bottom cell. (Make sure to scroll underneath the cell if you can't see all of them)


In [11]:
from google.colab import auth
import pandas as pd
import numpy as np
auth.authenticate_user()
import gspread
from google.auth import default
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
from google.colab import auth
from IPython.display import clear_output
creds, _ = default()
drive.mount('/content/drive/')
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount("mnt")
!pip install --upgrade gspread
import gspread
from datetime import date
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px

sheet_id='1zBvZf07H6HrQNsCRNs2-d22T31KnHi65m5iZbzLgxrY'

sh = gc.open_by_key(sheet_id)
sh.worksheets()

!pip install pandasql
import sqlite3
clear_output()

The code block below here reads the data into this colab sheet. Just run it

In [12]:
def WorksheetLoading(Name):
  Data_Worksheet=sh.worksheet(Name)
  Data_Worksheet=Data_Worksheet.get_all_values()
  headers1 = Data_Worksheet.pop(0)
  AllEmployeeData=pd.DataFrame(Data_Worksheet, columns=headers1)
  return AllEmployeeData

All_Courses_Data=WorksheetLoading('Course A')
#All_Courses_Data.drop('Location')
All_Courses_Data

,Email,Status,Percentage Complete,Course Name,Employee_Name,Manager?,Full_Name,Job,Supervisor_Name,Department,Sub_Department,Location
0,dave.roycroft@contentstack.com,Not Started,0%,Accelerate SKO Pre-work v.1,Dave Roycroft,Y,"Roycroft, David",Dir Learning Development,Sherry R. Holland,General and Admin,GA - People Team,Florida
1,elizabeth.himelick@contentstack.com,In Progress,63%,Accelerate SKO Pre-work v.1,Elizabeth Himelick,N,"Himelick, Elizabeth A.",Sr Account Executive,Christopher M. McCann,Sales,S - Sales General,Texas
2,guiomar.andrade@contentstack.com,In Progress,13%,Accelerate SKO Pre-work v.1,Guiomar Andrade,N,"Andrade, Guiomar",Sr Account Executive,William Van Horen,Sales,S - Sales General,France
3,lloyd.layton@contentstack.com,Pending Review,88%,Accelerate SKO Pre-work v.1,Lloyd Layton,N,"Layton, Lloyd W.",Sr Account Executive,Rachel Connors,Sales,S - Sales General,Connecticut
4,katelin.bishop@contentstack.com,Pending Review,100%,Accelerate SKO Pre-work v.1,Katelin Bishop,N,"Bishop, Katelin",Account Executive,Rachel Connors,Sales,S - Sales General,Massachusetts
...,...,...,...,...,...,...,...,...,...,...,...,...
1754,deepak.chaturvedi@contentstack.com,In Progress,40%,Unit Testing and Test Driven Development v.1,Deepak Chaturvedi,N,"Chaturvedi, Deepak",Assoc Software Eng Intern,Naveen P. DSA,Product Development,PD - Engineering,India Office
1755,abhay.kumar@contentstack.com,Not Started,0%,Unit Testing and Test Driven Development v.1,Abhay Kumar,N,"Kumar, Abhay S.",Sr CSE I T2,Saqlain T. Kali,Product Development,PD - Customer Support,India Office
1756,rohit.kini@contentstack.com,Not Started,0%,Unit Testing and Test Driven Development v.1,ROHIT KINI,N,"Kini, Rohit B.",Software Engineer II,Vikrant J. Raut,Sales,S - Tech Services Org,India Office
1757,dave.roycroft@contentstack.com,In Progress,18%,Working with Automation Hub - For Consideratio...,Dave Roycroft,Y,"Roycroft, David",Dir Learning Development,Sherry R. Holland,General and Admin,GA - People Team,Florida


Run the 2 code blocks below to call the graphs

In [13]:
All_Courses_Data['Percentage Complete'] = pd.to_numeric(All_Courses_Data['Percentage Complete'].str.replace('%', ''))
average=All_Courses_Data['Percentage Complete'].unique()

Department bubble graphs

In [14]:
# Department Graphs
hex_colors = ['#EB5646', '#7C4DFF', '#5A20B9', '#2A0F57', '#000']

pivot_df = All_Courses_Data.pivot_table(index='Department', columns='Status', aggfunc='size', fill_value=0)
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df = pivot_df.sort_values(by='Total')

fig = go.Figure()

statuses = pivot_df.columns[:-1]
num_statuses = len(statuses)
for i, status in enumerate(statuses):
    color = hex_colors[i % len(hex_colors)]
    fig.add_trace(go.Scatter(
        x=pivot_df.index,
        y=[i] * len(pivot_df),
        mode='markers',
        marker=dict(
            color=color,
            symbol='circle',
            size=pivot_df[status],
            sizemode='diameter',
            sizeref=1.5,
            sizemin=1
        ),
        name=status,
    ))

fig.update_layout(title='Course Status by Department', title_x=0.5)
fig.update_layout(
    font=dict(
        size=10
    )
)
fig.update_layout(
    xaxis=dict(
        title='Department Name',
        showticklabels=True
    ),
    yaxis=dict(
        title='Status',
        showticklabels=True,
        tickmode='array',
        tickvals=list(range(num_statuses)),
        ticktext=statuses
    ),
    height=600,
    width=1000
)
fig.show()

In [15]:
#Sub Department Graph
pivot_df = All_Courses_Data.pivot_table(index='Sub_Department', columns='Status', aggfunc='size', fill_value=0)
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df = pivot_df.sort_values(by='Total')
fig = go.Figure()

statuses = pivot_df.columns[:-1]
num_statuses = len(statuses)

for i, status in enumerate(statuses):
    color = hex_colors[i % len(hex_colors)]

    fig.add_trace(go.Scatter(
        x=pivot_df.index,
        y=[i] * len(pivot_df),
        mode='markers',
        marker=dict(
            color=color,
            symbol='circle',
            size=pivot_df[status],
            sizemode='diameter',
            sizeref=1.5,
            sizemin=1
        ),
        name=status,
    ))

fig.update_layout(title='Course Status by Sub Department', title_x=0.5)
fig.update_layout(
    font=dict(
        size=10
    )
)

fig.update_layout(
    xaxis=dict(
        title='Sub Department Name',
        showticklabels=True
    ),
    yaxis=dict(
        title='Status',
        showticklabels=True,
        tickmode='array',
        tickvals=list(range(num_statuses)),
        ticktext=statuses
    ),
    height=600,
    width=1000
)
fig.show()

In [16]:
#status for each unique location
grouped = All_Courses_Data.groupby(['Location', 'Status']).size().reset_index(name='Count')
def group_states(location):
    if location in ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Pennsylvania 1', "Pennsylvania 2", 'Pennsylvania 3', 'Ohio 1', 'Ohio 2', 'Rhode Island', 'Tennesse', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'United Kingdom','Netherlands','Luxembourg','Iceland','France','Brazil','Australia','Türkiye']:
        return 'US/EMEA/ROW Only'
    else:
        return location
grouped['Location'] = grouped['Location'].apply(group_states)
grouped['Location'] = grouped['Location'].replace({'India Office': 'India Office', 'US/EMEA/ROW Only': 'US/EMEA/ROW Only'})
pivot_df = grouped.pivot_table(index='Status', columns='Location', values='Count', aggfunc='sum').fillna(0)
fig = go.Figure()
color_scheme = ['#EB5646', '#7C4DFF']
fill_opacity = 0.5
for i, column in enumerate(['India Office', 'US/EMEA/ROW Only']):
    fill_color_rgba = f'rgba({int(color_scheme[i][1:3], 16)},{int(color_scheme[i][3:5], 16)},{int(color_scheme[i][5:], 16)},{fill_opacity})'
    fig.add_trace(go.Scatter(x=pivot_df.index, y=pivot_df[column], mode='lines+markers',
                             name=column, line=dict(color=color_scheme[i]),
                             fill='tozeroy', fillcolor=fill_color_rgba))

fig.update_layout(title='Status for each Unique Location', title_x=0.5)
fig.update_xaxes(title_text='Status')
fig.update_yaxes(title_text='Count')
fig.update_layout(legend=dict(y=0.5, traceorder='reversed'))
fig.update_layout(height=400, width=1000)
fig.show()


In [17]:
#Manager? vs Status Graph
grouped = All_Courses_Data.groupby(['Manager?', 'Status']).size().reset_index(name='Count')
pivot_df = grouped.pivot(index='Status', columns='Manager?', values='Count')
x_labels = pivot_df.index
y_values_y = pivot_df['Y'].values
y_values_n = pivot_df['N'].values
fig = go.Figure()
color_scheme = ['#EB5646', '#7C4DFF']
fill_opacity = 0.5
fig.add_trace(go.Scatter(
    x=x_labels, y=y_values_y, mode='lines+markers', name='Manager',
    line=dict(color=color_scheme[0], width=4),
    fill='tozeroy', fillcolor=f'rgba({int(color_scheme[0][1:3], 16)},{int(color_scheme[0][3:5], 16)},{int(color_scheme[0][5:], 16)},{fill_opacity})',
    marker=dict(color=color_scheme[0], size=8)
))

fig.add_trace(go.Scatter(
    x=x_labels, y=y_values_n, mode='lines+markers', name='Employee',
    line=dict(color=color_scheme[1], width=4),
    fill='tozeroy', fillcolor=f'rgba({int(color_scheme[1][1:3], 16)},{int(color_scheme[1][3:5], 16)},{int(color_scheme[1][5:], 16)},{fill_opacity})',
    marker=dict(color=color_scheme[1], size=8)
))

fig.update_layout(title='Statuses of Managers vs Employees', title_x=0.5)
fig.update_xaxes(title_text='Status')
fig.update_yaxes(title_text='Count')
fig.update_layout(legend=dict(y=0.5, traceorder='reversed'))
fig.update_layout(height=400, width=1000)
fig.show()


In [18]:
#Plotting average percent completed per graph
All_Courses_Data['Percentage Complete'] = pd.to_numeric(All_Courses_Data['Percentage Complete'])
average_values = All_Courses_Data.groupby('Course Name')['Percentage Complete'].mean().reset_index()
average_values.sort_values(by='Percentage Complete', inplace=True)
fig = px.scatter(average_values, x='Percentage Complete', y='Course Name',
                 title='Average Values for Each Course', color='Course Name')
fig.update_layout(
    xaxis_title='Average Value',
    yaxis_title='Course Name',
    yaxis_categoryorder='total ascending',
    xaxis_tickangle=0,
    height=1200,
    width=1600,
)

colors = ['#EB5646'] * len(average_values)
fig.update_traces(marker=dict(color=colors))
fig.show()

In [19]:
#course status by supervisor name

hex_colors = ['#EB5646', '#2A0F57', '#5A20B9', '#7C4DFF', '#0E0E0F']

pivot_df = All_Courses_Data.pivot_table(index='Supervisor_Name', columns='Status', aggfunc='size', fill_value=0)
fig = go.Figure()

statuses = pivot_df.columns[:-1]
num_statuses = len(statuses)

for i, status in enumerate(statuses):
    color = hex_colors[i % len(hex_colors)]  # Use modulus to cycle through the colors
    fig.add_trace(go.Bar(
        x=pivot_df.index,
        y=pivot_df[status],
        name=status,
        marker=dict(color=color),
        offsetgroup=i,
    ))

fig.update_layout(title='Course Status by Supervisor Name', title_x=0.5)
fig.update_layout(
    font=dict(
        size=10
    )
)

fig.update_layout(
    barmode='relative',
    xaxis_title='Supervisor Name',
    yaxis_title='Count',
    height=500,
    width=2500
)

fig.show()

In [20]:
hex_colors = ['#EB5646', '#2A0F57', '#5A20B9', '#7C4DFF', '#0E0E0F']

pivot_df = All_Courses_Data.pivot_table(index='Supervisor_Name', columns='Status', aggfunc='size', fill_value=0)
fig = go.Figure()

statuses = pivot_df.columns
num_statuses = len(statuses)

# Calculate total counts per supervisor
pivot_df['Total'] = pivot_df.sum(axis=1)

for i, status in enumerate(statuses):
    color = hex_colors[i % len(hex_colors)]

    # Calculate percentage values
    pivot_df[f'{status}_percentage'] = (pivot_df[status] / pivot_df['Total']) * 100

    fig.add_trace(go.Bar(
        x=pivot_df.index,
        y=pivot_df[f'{status}_percentage'],
        name=status,
        marker=dict(color=color),
        offsetgroup=i,
    ))

fig.update_layout(title='Stacked Percentage Bar Graph: Course Status by Supervisor Name', title_x=0.5)
fig.update_layout(
    font=dict(
        size=10
    )
)

fig.update_layout(
    barmode='relative',
    xaxis_title='Supervisor Name',
    yaxis_title='Percentage',
    height=500,
    width=3000
)

fig.show()

In [21]:
#course status by course name

pivot_df = All_Courses_Data.pivot_table(index='Course Name', columns='Status', aggfunc='size', fill_value=0)
pivot_df['Total'] = pivot_df.sum(axis=1)
for column in pivot_df.columns:
    pivot_df[column] = (pivot_df[column] / pivot_df['Total']) * 100
pivot_df = pivot_df.sort_values(by='Completed', ascending=False)
fig = go.Figure()
statuses = pivot_df.columns[:-1]
for i, status in enumerate(statuses):
    color = hex_colors[i % len(hex_colors)]
    fig.add_trace(go.Bar(
        y=pivot_df.index,
        x=pivot_df[status],
        name=status,
        orientation='h',
        marker=dict(color=color),
        offsetgroup=i,
    ))

fig.update_layout(title='Course Status by Course Name', title_x=0.5)
fig.update_layout(
    font=dict(
        size=10
    )
)

fig.update_layout(
    barmode='stack',
    xaxis_title='Percentage',
    yaxis_title='Course Name',
    height=600,
    width=1100,
)

fig.show()

In [24]:
pivot_df = All_Courses_Data.pivot_table(index='Employee_Name', columns='Status', aggfunc='size', fill_value=0)
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df.sort_values(by='Total', ascending=True, inplace=True)
hex_colors = ['#0E0E0F', '#2A0F57', '#5A20B9', '#7C4DFF', '#EB5646']  # Use the same colors as in the second graph

fig = go.Figure()
statuses = pivot_df.columns[:-1]
num_statuses = len(statuses)

for i, status in enumerate(statuses):
    color = hex_colors[i % len(hex_colors)]
    fig.add_trace(go.Bar(
        x=pivot_df.index,
        y=pivot_df[status],
        name=status,
        marker=dict(color=color),
        offsetgroup=i,
    ))

fig.update_layout(title='Course Status by Employee Name', title_x=0.5)
fig.update_layout(
    font=dict(
        size=10
    )
)

fig.update_layout(
    barmode='relative',
    xaxis_title='Employee_Name',
    yaxis_title='Count',
    height=1000,
    width=6000,
)

fig.show()